Nota: Esse notebook não é para ser rodado, serve apenas como uma forma mais organizada de mostrar a evolução das funções testadas pelo método RED/GREEN

## get_elements function

In [ ]:
import unittest
from datetime import datetime, timedelta
from tests.test_users.mocks import Schedule, ScheduleManagement, Element, ElementManagement
from src.user.user_model import *

RED 1:

In [ ]:
def test_get_all_elements(self):
        # Test getting all element ids from user schedules, without repetition
        user = User("id", "username", "email", ["id1", "id2"])
        result = user.get_elements()        
        self.assertEqual(sorted(result), ['elementid1', 'elementid2', 
                                  'elementid3', 'elementid4', 'elementid5'])

In [ ]:
def get_elements(self, schedules: list=None) -> list:
        '''
        returns a list of ids of elements of all schedules (or only specific ones) 
        that the user is a part of  
        '''
        elements = []
        for schedule in schedules:
            schedule = ScheduleManagement().get_schedule(schedule)
            elements += schedule.get_elements()

        return elements

GREEN 1:

In [ ]:
def get_elements(self, schedules: list=None) -> list:
    '''
    returns a list of ids of elements of all schedules (or only specific ones) 
    that the user is a part of  
    '''
    elements = []
    for schedule in schedules:
        schedule = ScheduleManagement().get_schedule(schedule)
        elements += schedule.get_elements()

    elements = list(set(elements)) # remove duplicates
    return elements

REFACTOR 1 (Para tratar classes Management como singletons):

In [ ]:
def get_elements(self, schedules: list=None) -> list:
    '''
    returns a list of ids of elements of all schedules (or only specific ones) 
    that the user is a part of  
    '''
    elements = []
    for schedule in schedules:
        schedule = self.schedule_management.get_schedule(schedule)
        elements += schedule.get_elements()

    elements = list(set(elements)) # remove duplicates
    return elements

RED 2

In [ ]:
def test_get_filtered_elements(self):
        # Test getting all element ids from user schedules, with filter
        user = User("id", "username", "email", ["id1", "id2", "id3"])
        result = user.get_elements(["id1", "id3"])        
        self.assertEqual(sorted(result), ['elementid1', 'elementid2', 
                                        'elementid5', 'elementid6', 'elementid7'])

In [ ]:
def get_elements(self, schedules: list=None) -> list:
    '''
    returns a list of ids of elements of all schedules (or only specific ones) 
    that the user is a part of  
    '''
    elements = []
    for schedule in schedules:
        schedule = self.schedule_management.get_schedule(schedule)
        elements += schedule.get_elements()

    elements = list(set(elements)) # remove duplicates
    return elements

GREEN 2

In [ ]:
def get_elements(self, schedules: list=None) -> list:
    '''
    returns a list of ids of elements of all schedules (or only specific ones) 
    that the user is a part of  
    '''
    if not schedules:
        schedules = self.schedules

    elements = []
    for schedule in schedules:
        schedule = self.schedule_management.get_schedule(schedule)
        elements += schedule.get_elements()

    elements = list(set(elements)) # remove duplicates
    return elements

RED 3

In [ ]:
def test_get_elements_from_schedule_user_isnt_in(self):
    # Test getting all element ids from a nonexistent schedule
    user = User("id", "username", "email", ["id1", "id2"])
    with self.assertRaises(UserNotInSchedule) as context:
        user.get_elements(["id3"])
    self.assertEqual(str(context.exception), 'Usuário não está nessa agenda: id3')

GREEN 3:

In [ ]:
def get_elements(self, schedules: list=None) -> list:
        '''
        Get all elements from the user schedules, without repetition, or
        from a list of filtered schedules

        Args:
            schedules: list of schedules ids

        Returns:
            A list of elements ids the user is a part of
        '''
        if not schedules:
            schedules = self.get_schedules()
        else:
            for schedule in schedules:
                if schedule not in self.get_schedules():
                    raise UserNotInSchedule(
                        f"Usuário não está nessa agenda: {schedule}")

        elements = []
        for schedule in schedules:
            schedule = self.schedule_management.get_schedule(schedule)
            elements += schedule.get_elements()

        elements = list(set(elements))
        return elements

### Função set_mail:

GREEN 1:

In [ ]:
def test_set_email(self):
        # Test setting an email
        user = User("id", "username", "email", ["id1", "id2"])
        user.set_email("new_email")
        self.assertEqual(user.email, "new_email")

In [ ]:
def set_email(self, email: str):
        """
        Set the user name

        Args:
            username: user name
        """
        self.email = email

RED 2:

In [ ]:
def test_set_email_with_trailing_space(self):
        # Test setting an email with trailing space
        user = User("id", "username", "email", ["id1", "id2"])
        user.set_email("new_email ")
        self.assertEqual(user.email, "new_email")

GREEN 2:

In [ ]:
def set_email(self, email: str):
        """
        Set the user name

        Args:
            username: user name
        """
        self.email = email.strip()

RED 3:

In [ ]:
def test_set_email_with_int(self):
        # Test setting an email with trailing space
        user = User("id", "username", "email", ["id1", "id2"])
        with self.assertRaises(TypeError) as context:
            user.set_email(123)
        self.assertEqual(str(context.exception), 
                         "O email deve ser uma string")

GREEN 3:

In [ ]:
def set_email(self, email: str):
        """
        Set the user name

        Args:
            username: user name
        """
        if type(email) != str:
            raise TypeError("O email deve ser uma string")
        else:
            self.email = email.strip()

RED 4:

In [ ]:
def test_set_blank_email(self):
        # Test setting a blank email
        user = User("id", "username", "email", ["id1", "id2"])
        with self.assertRaises(EmailCantBeBlank) as context:
            user.set_email("")
        self.assertEqual(str(context.exception), 
                         "O email não pode ser vazio")

GREEN 4:

In [ ]:
def set_email(self, email: str):
    """
    Set the user name

    Args:
        username: user name
    """
    if type(email) != str:
        raise TypeError("O email deve ser uma string")
    elif email == "":
        raise EmailCantBeBlank("O email não pode ser vazio")
    else:
        self.email = email.strip()

### Check_disponibility():

Red 1

In [ ]:
def test_check_disponibility(self):
        user = User("id", "username", "email", ["id1", "id2"])
        time = (datetime.now() + timedelta(hours=2), 
                datetime.now() + timedelta(hours=3))
        result = user.check_disponibility(time)
        self.assertTrue(result)

Green 1

In [ ]:
def check_disponibility(self, time: tuple) -> bool:
    """
    Checks if the user is available at a given time, based on the user's
    schedules and elements. It should not raise a conflict if the type
    of the element is not 'evento'.

    Args:
        time: tuple with the start and end time to be checked

    Returns:
        True if the user is available, False otherwise
    """
    element_ids = self.get_elements()
    element_management = ElementManagement.get_instance()

    for element_id in element_ids:
        element = element_management.get_element(element_id)
        
        # Check if the start time of the element is within the given time period
        if time[0] <= element.start_time <= time[1]:
            return False
        
        # Check if the end time of the element is within the given time period
        if time[0] <= element.end_time <= time[1]:
            return False
        
        # Check if the given time period is within the start 
        # and end time of the element
        if element.start_time <= time[0] <= element.end_time or \
            element.start_time <= time[1] <= element.end_time:
            return False
    
    return True

RED 2:

In [ ]:
def test_check_disponibility_end_time_same_as_other_event_start_time(self):
        user = User("id", "username", "email", ["id1", "id2"])
        time = (datetime.now() + timedelta(hours=11), 
                self.ElementManagement.get_element("elementid5").start_time)
        result = user.check_disponibility(time)
        self.assertTrue(result)

GREEN 2:

In [ ]:
def check_disponibility(self, time: tuple) -> bool:
    """
    Checks if the user is available at a given time, based on the user's
    schedules and elements. It should not raise a conflict if the type
    of the element is not 'evento'.

    Args:
        time: tuple with the start and end time to be checked

    Returns:
        True if the user is available, False otherwise
    """
    element_ids = self.get_elements()
    element_management = ElementManagement.get_instance()

    for element_id in element_ids:
        element = element_management.get_element(element_id)
        
        # Check if the start time of the element is within the given time period
        if time[0] <= element.start_time < time[1]:
            return False
        
        # Check if the end time of the element is within the given time period
        if time[0] < element.end_time <= time[1]:
            return False
        
        # Check if the given time period is within the start 
        # and end time of the element
        if element.start_time <= time[0] < element.end_time or \
            element.start_time < time[1] <= element.end_time:
            return False
    
    return True

RED 3:

In [ ]:
def test_check_disponibility_ignoring_non_event_elements(self):
        user = User("id", "username", "email", ["id1", "id2"])
        time = (self.ElementManagement.get_element("elementid3").start_time,
                self.ElementManagement.get_element("elementid3").end_time)
        result = user.check_disponibility(time)
        self.assertTrue(result)

GREEN 3:

In [ ]:
def check_disponibility(self, time: tuple) -> bool:
        """
        Checks if the user is available at a given time, based on the user's
        schedules and elements. It should not raise a conflict if the type
        of the element is not 'evento'.

        Args:
            time: tuple with the start and end time to be checked

        Returns:
            True if the user is available, False otherwise
        """
        element_ids = self.get_elements()
        element_management = ElementManagement.get_instance()

        for element_id in element_ids:
            element = element_management.get_element(element_id)
            if element.type != 'evento':
                continue
            
            # Check if the start time of the element is within the given time period
            if time[0] <= element.start_time < time[1]:
                return False
            
            # Check if the end time of the element is within the given time period
            if time[0] < element.end_time <= time[1]:
                return False
            
            # Check if the given time period is within the start 
            # and end time of the element
            if element.start_time <= time[0] < element.end_time or \
                element.start_time < time[1] <= element.end_time:
                return False
        
        return True

RED 4:

In [ ]:
def test_check_disponibility_input_type_exception(self):
        user = User("id", "username", "email", ["id1", "id2"])
        time = "time"
        with self.assertRaises(TypeError) as context:
            user.check_disponibility(time)
        self.assertEqual(str(context.exception), 
                         "O horário deve ser uma tupla")

GREEN 4:

In [ ]:
def check_disponibility(self, time: tuple) -> bool:
        """
        Checks if the user is available at a given time, based on the user's
        schedules and elements. It should not raise a conflict if the type
        of the element is not 'evento'.

        Args:
            time: tuple with the start and end time to be checked

        Returns:
            True if the user is available, False otherwise
        """
        if type(time) != tuple:
            raise TypeError("O horário deve ser uma tupla")

        element_ids = self.get_elements()
        element_management = ElementManagement.get_instance()

        for element_id in element_ids:
            element = element_management.get_element(element_id)
            if element.type != 'evento':
                continue
            
            # Check if the start time of the element is within the given time period
            if time[0] <= element.start_time < time[1]:
                return False
            
            # Check if the end time of the element is within the given time period
            if time[0] < element.end_time <= time[1]:
                return False
            
            # Check if the given time period is within the start 
            # and end time of the element
            if element.start_time <= time[0] < element.end_time or \
                element.start_time < time[1] <= element.end_time:
                return False
        
        return True